In [ ]:
# !pip install openmeteo-requests
# !pip install requests-cache retry-requests numpy pandas
# !pip install geopandas shapely

Sử dụng file geojson để centroid dựa trên các polygon

In [47]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
import time
from datetime import datetime, timedelta, timezone

In [24]:
import geopandas as gpd

# Đọc file GeoJSON
gdf = gpd.read_file("N:\\workspace\\DSR301m\\git_project\\data\\diaphanhuyen.geojson")

# Lọc riêng Hà Nội (điều chỉnh theo cột của bạn)
hanoi = gdf[gdf["Ten_Tinh"] == "Hà Nội"].copy()

# --- Chuyển sang hệ tọa độ phẳng (UTM zone 48N cho Hà Nội)
hanoi = hanoi.to_crs(epsg=32648)

# Tính centroid trong hệ tọa độ phẳng
centroids = hanoi.geometry.centroid

# Lấy tọa độ (sau đó chuyển ngược về WGS84)
centroids_wgs84 = centroids.to_crs(epsg=4326)
hanoi["lon"] = centroids_wgs84.x
hanoi["lat"] = centroids_wgs84.y

# --- Xóa cột centroid phụ, chỉ giữ geometry gốc
# (nếu muốn giữ cả centroid WKT thì thêm dòng dưới)
# hanoi["centroid_wkt"] = centroids_wgs84.to_wkt()

# Lưu lại file GeoJSON mới
hanoi.to_file("hanoi_with_centroid.geojson", driver="GeoJSON")

# In kết quả kiểm tra
print(hanoi[["Ten_Huyen", "lon", "lat"]].head())


     Ten_Huyen         lon        lat
415    Ba Dinh  105.828509  21.039593
416   Cau Giay  105.788032  21.028212
419  Long Bien  105.894906  21.042786
422   Hoai Duc  105.695768  21.025088
424     Tay Ho  105.817464  21.071490


In [26]:
hanoi[["Ten_Huyen", "lon", "lat"]].to_csv("centroid_hanoi.csv", index=False)

In [44]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"


In [48]:
# --- Lấy thời gian hiện tại (theo giờ Việt Nam) ---
now = datetime.now(timezone.utc) + timedelta(hours=7)
end_date = now.strftime("%Y-%m-%d")      # API chỉ nhận theo ngày
start_date = (now - timedelta(days=7)).strftime("%Y-%m-%d")  # ví dụ lấy 7 ngày gần nhất


# --- DataFrame các quận/huyện ---
tub_huyen = hanoi[["Ten_Huyen", "lon", "lat"]]

all_data = []

for _, row in tub_huyen.iterrows():
    name = row["Ten_Huyen"]
    lat = row["lat"]
    lon = row["lon"]

    print(f"📡 Lấy dữ liệu cho {name} ({lat:.4f}, {lon:.4f})...")

    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "wind_speed_10m",
            "cloud_cover",
            "rain",
        ],
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "Asia/Bangkok",
    }

    try:
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        hourly = response.Hourly()
        times = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ).tz_convert("Asia/Bangkok")

        # Cắt dữ liệu đến đúng giờ hiện tại (nếu API trả dư)
        df = pd.DataFrame({
            "Ten_Huyen": name,
            "date": times,
            "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
            "relative_humidity_2m": hourly.Variables(1).ValuesAsNumpy(),
            "wind_speed_10m": hourly.Variables(2).ValuesAsNumpy(),
            "cloud_cover": hourly.Variables(3).ValuesAsNumpy(),
            "rain": hourly.Variables(4).ValuesAsNumpy(),
        })
        df = df[df["date"] <= now]  # chỉ giữ đến giờ hiện tại

        all_data.append(df)
        time.sleep(2)

    except Exception as e:
        print(f"⚠️ Lỗi với {name}: {e}")

weather_df = pd.concat(all_data, ignore_index=True)
weather_df = weather_df.sort_values(by=["Ten_Huyen", "date"])
weather_df.to_csv("weather_hanoi_latest.csv", index=False)

print("✅ Hoàn tất! Dữ liệu mới nhất đã được lưu vào weather_hanoi_latest.csv")

📡 Lấy dữ liệu cho Ba Dinh (21.0396, 105.8285)...
📡 Lấy dữ liệu cho Cau Giay (21.0282, 105.7880)...
📡 Lấy dữ liệu cho Long Bien (21.0428, 105.8949)...
📡 Lấy dữ liệu cho Hoai Duc (21.0251, 105.6958)...
📡 Lấy dữ liệu cho Tay Ho (21.0715, 105.8175)...
📡 Lấy dữ liệu cho Thach That (21.0188, 105.5304)...
📡 Lấy dữ liệu cho Tu Liem (21.0501, 105.7552)...
📡 Lấy dữ liệu cho Gia Lam (21.0321, 105.9482)...
📡 Lấy dữ liệu cho Dan Phuong (21.1135, 105.6732)...
📡 Lấy dữ liệu cho Phuc Tho (21.1135, 105.5809)...
📡 Lấy dữ liệu cho Son Tay (21.1031, 105.4752)...
📡 Lấy dữ liệu cho Dong Anh (21.1392, 105.8353)...
📡 Lấy dữ liệu cho Phu Xuyen (20.7361, 105.8968)...
📡 Lấy dữ liệu cho Ung Hoa (20.7114, 105.7969)...
📡 Lấy dữ liệu cho My Duc (20.6914, 105.7164)...
📡 Lấy dữ liệu cho Thuong Tin (20.8450, 105.8703)...
📡 Lấy dữ liệu cho Thanh Oai (20.8624, 105.7736)...
📡 Lấy dữ liệu cho Chuong My (20.8778, 105.6597)...
📡 Lấy dữ liệu cho Thanh Tri (20.9371, 105.8355)...
📡 Lấy dữ liệu cho Ha Dong (20.9612, 105.7532)...

Code hoàn chỉnh theo ý tưởng incremental update:
1. Kiễm tra xem file đó có tồn tại hay không? Nếu không thì lấy data 7 ngày gần nhất.
2. Nếu có thì viết tiếp vào data hiện tại

In [ ]:
import time
import pandas as pd
from datetime import datetime, timedelta, timezone
import openmeteo_requests
from retry_requests import retry
import requests_cache
import os

# --- Setup cache & retry ---
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=3, backoff_factor=0.3)
openmeteo = openmeteo_requests.Client(session=retry_session)
url = "https://api.open-meteo.com/v1/forecast"

# --- File CSV đầu ra ---
output_file = "weather_hanoi_latest.csv"

# --- DataFrame các quận/huyện ---
# Giả sử bạn có tub_huyen = hanoi[["Ten_Huyen", "lon", "lat"]]

# --- Lấy thời gian hiện tại (theo giờ VN) ---
now = datetime.now(timezone.utc) + timedelta(hours=7)
end_date = now.strftime("%Y-%m-%d")

# Nếu file cũ đã tồn tại → đọc để xác định start_date mới
if os.path.exists(output_file):
    old_data = pd.read_csv(output_file, parse_dates=["date"])
    last_time = old_data["date"].max()
    last_time = last_time.tz_localize("Asia/Bangkok") if last_time.tzinfo is None else last_time
    start_date = last_time.strftime("%Y-%m-%d")
    print(f"🔁 Phát hiện file cũ. Lấy dữ liệu từ {start_date} → {end_date}")
else:
    old_data = pd.DataFrame()
    start_date = (now - timedelta(days=7)).strftime("%Y-%m-%d")
    print(f"🆕 Lần đầu chạy. Lấy dữ liệu từ {start_date} → {end_date}")

all_data = []

for _, row in tub_huyen.iterrows():
    name = row["Ten_Huyen"]
    lat = row["lat"]
    lon = row["lon"]

    print(f"📡 Gọi API cho {name} ({lat:.4f}, {lon:.4f})...")

    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "wind_speed_10m",
            "cloud_cover",
            "rain",
        ],
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "Asia/Bangkok",
    }

    try:
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        hourly = response.Hourly()
        times = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ).tz_convert("Asia/Bangkok")

        df = pd.DataFrame({
            "Ten_Huyen": name,
            "date": times,
            "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
            "relative_humidity_2m": hourly.Variables(1).ValuesAsNumpy(),
            "wind_speed_10m": hourly.Variables(2).ValuesAsNumpy(),
            "cloud_cover": hourly.Variables(3).ValuesAsNumpy(),
            "rain": hourly.Variables(4).ValuesAsNumpy(),
        })

        # Cắt đến thời điểm hiện tại
        df = df[df["date"] <= now]
        all_data.append(df)
        time.sleep(1)

    except Exception as e:
        print(f"⚠️ Lỗi với {name}: {e}")

# --- Gộp tất cả quận/huyện ---
new_data = pd.concat(all_data, ignore_index=True)
new_data = new_data.sort_values(by=["Ten_Huyen", "date"])

# --- Nếu có dữ liệu cũ thì append phần mới ---
if not old_data.empty:
    combined = pd.concat([old_data, new_data], ignore_index=True)
    combined = combined.drop_duplicates(subset=["Ten_Huyen", "date"])  # tránh trùng
else:
    combined = new_data

combined = combined.sort_values(by=["Ten_Huyen", "date"])
combined.to_csv(output_file, index=False)

print(f"✅ Hoàn tất cập nhật! Đã lưu {output_file}")
